In [1]:
import os
import numpy
import random
import tensorflow.keras as keras
from keras import layers
from itertools import count
from gui.puzzle_class import Twisty_Puzzle
from gui.ai_modules.ai_data_preparation import state_for_ai

def action_num (puzzle, action):
    for i, act in zip(count(), puzzle.moves.keys()):
        if action == act :
            return i

def action_from_num (puzzle, num):
    list(puzzle.moves.values())[num]

def prepare_data (puzzle, q_table):
    training_table = dict()
    output_template = [0 for _ in range(len(puzzle.moves))]
    for state_action , value in q_table.items() :
        (state , action) = state_action
        state += tuple(state_for_ai(puzzle.SOLVED_STATE)[0])
        inner = training_table.get(state) or output_template.copy()
        inner[action_num(puzzle, action)] = value
        training_table[state] = inner
    return list(training_table.keys()) , list(training_table.values())

def import_q_table (filename="floppy_q_table.txt"):
    q_table = dict()
    with open(filename, "r") as file:
        q_table = eval (file.read())
    return q_table

<IPython.core.display.Javascript object>

In [2]:
def initialize_nn (puzzle):
    actions = game_object.moves
    solved_state = state_for_ai(game_object.SOLVED_STATE)[0]
    input_size = len(solved_state)*2
    output_size = len(actions)
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_size)))
    model.add(layers.Dense((input_size+output_size)/2))
    model.add(layers.Dense(output_size))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [3]:
def train_nn (model, samples, labels, epochs=100, batch_size=30):
    return model.fit(samples, labels, epochs=epochs, batch_size=batch_size,
                     use_multiprocessing=True, verbose=False)

In [4]:
puzzle = Twisty_Puzzle()
puzzle.load_puzzle("floppy_cube")

q_table = import_q_table()
inputs, outputs = prepare_data(puzzle, q_table)
model = initialize_nn(puzzle)
history = train_nn(model, inputs, outputs)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ai_state, color_list = state_for_ai(puzzle.SOLVED_STATE)
reward_dict = {"solved":1, "timeout":-1, "move":-0.02}
AI_class = puzzle_ai(deepcopy(puzzle.moves), ai_state, reward_dict=reward_dict, name=puzzle.PUZZLE_NAME)

def nn_get_greedy_move (model, puzzle):
    state = state_for_ai(puzzle.vpy_objects)[0] \
        + state_for_ai(puzzle.SOLVED_STATE)[0]
    predictions = list(model.predict(state))
    max_value = None
    actions = []
    for action, value in enumerate(predictions):
        if not max_value or value > max_value:
            actions = [action]
            max_value = value
        elif value == max_value:
            actions.append(action)
    return action_from_num(random.choice(actions))

def nn_get_epsilon_greedy_move (model, puzzle, epsilon=0.5):
    if random.random() > epsilon:
        return nn_get_greedy_move(model, puzzle)
    else:
        return random.choice(list(puzzle.moves.values()))

def nn_solve_puzzle (model, puzzle, max_moves, epsilon=0):
    while max_moves >= 0:
        if state_for_ai(puzzle.SOLVED_STATE)[0] == state_for_ai(puzzle.vpy_objects)[0]:
            return True
        else :
            puzzle.perform_move(nn_get_epsilon_greedy_move(model, puzzle, epsilon=epsilon))
            max_moves -= 1
    return False